In [207]:
import os
import re
import json
import camel_tools
import nltk
import pickle
import jsonlines
import pandas as pd
from pathlib import Path
import qalsadi.lemmatizer
from camel_tools.ner import NERecognizer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar, dediac_bw

In [208]:
# Instantiate the lemmatizer
lemmer = qalsadi.lemmatizer.Lemmatizer()

# Read the JSON Lines file
df = pd.DataFrame()
with jsonlines.open('../Datasets/AIC Val/ds.jsonl') as reader:
    for line in reader:
        df = df.append(line, ignore_index=True)

# Define file paths
pass_pg_path = '../Debug/Token Tracking/Passed_Tokens/passed_paragraphs.txt'
pass_sum_path = '../Debug/Token Tracking/Passed_Tokens/passed_summaries.txt'
removed_pg_path = '../Debug/Token Tracking/Removed_Tokens/removed_paragraphs.txt'
removed_sum_path = '../Debug/Token Tracking/Removed_Tokens/removed_summaries.txt'
processed_records_path = "../Debug/Preprocessed Dataset/Processed_Dataset.csv"
processed_records_Qalpath = "../Debug/Preprocessed Dataset/QalProcessed_Dataset.csv"

# Load Arabic stopwords
with open(Path("../Preprocessing/Stopwords/Stopwords_List_HF.txt"), "r", encoding="utf-8") as f:
    arabic_stopwords = set(f.read().splitlines())

C:\Users\steve\AppData\Local\Temp\ipykernel_9480\70859954.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(line, ignore_index=True)
C:\Users\steve\AppData\Local\Temp\ipykernel_9480\70859954.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(line, ignore_index=True)
C:\Users\steve\AppData\Local\Temp\ipykernel_9480\70859954.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(line, ignore_index=True)
C:\Users\steve\AppData\Local\Temp\ipykernel_9480\70859954.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(line, ignore_index=True)
C:\Users\steve\AppData\Local\Temp\ipykernel_

In [209]:
class Preprocessor():
    
    def Tokenize(text):
        return simple_word_tokenize(text)

    def Lemmatize(text):
        return lemmer.lemmatize_text(' '.join(text))

    def DeDiacritics(text):
        return dediac_ar(text)

    def Remove_SW(tokens, stopwords):
        removed_tokens = []
        passed_tokens = []
        for token in tokens:
            if token in stopwords:
                removed_tokens.append(token)
            else:
                passed_tokens.append(token)
    
        return passed_tokens, removed_tokens

    def Remove_SW_Camel(tokens, stopwords):
        return [token for token in tokens if token not in stopwords]

    def Normalize_Numbers(text, type):
        if type == 1:
            text = re.sub('[0-9]', lambda match: chr(int(match.group(0)) + 1632), text)
        elif type == 2:
            text = re.sub('[١٢٣٤٥٦٧٨٩٠]', lambda match: str(ord(match.group(0)) - 1632), text)
        elif type == 3:
            text = re.sub('[0-9٠١٢٣٤٥٦٧٨٩]', '', text)
        return text

    def Normalize(text):
        text = re.sub("[إأٱآا]", "ا", text)
        text = re.sub("ى", "ي", text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        text = re.sub("[:“،؟!()»«{}؛]","", text)
        text = re.sub("%", "", text)
        text = re.sub('[٠١٢٣٤٥٦٧٨٩0123456789]', '', text)

        return text
    
    def Preprocess_Qalsadi(text):
        #Not done Library bug issue
        Ftokens = []
        sw = ["،",".",",","؛","ء","ءَ","آ","(",")","!",":","=","-",'"']
        tokens = Preprocessor.DeDiacritics(text)
        lemmatized_tokens = lemmer.lemmatize_text(tokens, return_pos=True)

        for token in lemmatized_tokens:
            if isinstance(token, tuple):
                word, tag = token
                if tag != 'stopword' and token not in sw:
                    Ftokens.append(word)
            else:
                if token != 'stopword' and token not in sw:
                    Ftokens.append(token)

        processed_text = ''.join(Ftokens)
        return processed_text
    
    def Default_Preprocess(text):
        tokens = Preprocessor.Normalize_Numbers(text, 1)
        tokens = Preprocessor.DeDiacritics(tokens)
        tokens = Preprocessor.Tokenize(tokens)
        tokens, rm_tokens = Preprocessor.Remove_SW(tokens, arabic_stopwords)
        tokens = Preprocessor.Lemmatize(tokens)
        return tokens,rm_tokens

    def Save_Passed(word, output):
        with open(output, 'a', encoding='utf-8') as file:
            file.write(word + '\n')
            
    def Save_Removed(word, output):
        with open(output, 'a', encoding='utf-8') as file:
            file.write(word + '\n')


In [210]:
open(pass_pg_path, 'w').close()
open(pass_sum_path, 'w').close()

open(removed_pg_path, 'w').close()
open(removed_sum_path, 'w').close()

processed_data = []
pkl_save = []
preprocessed_qalsadi = []
final_records_ct = 0

for index, row in df.iterrows():
    
    preprocessed_Paragraph, removed_Paragraph = Preprocessor.Default_Preprocess(row['paragraph'])
    
    processed_data.append({'paragraph': ' '.join(preprocessed_Paragraph), 'summary': row['summary']})

    # preprocessed_Paragraph_qalsadi = Preprocessor.Preprocess_Qalsadi(row['paragraph'])
    # preprocessed_Summary_qalsadi = Preprocessor.Preprocess_Qalsadi(row['summary'])
    
    # preprocessed_qalsadi.append({'paragraph': ' '.join(preprocessed_Paragraph_qalsadi), 'summary': ' '.join(preprocessed_Summary_qalsadi)})
    
    for word in preprocessed_Paragraph:
        Preprocessor.Save_Passed(word, pass_pg_path)

    for word in removed_Paragraph:
        Preprocessor.Save_Removed(word, removed_pg_path)
        
    # for word in preprocessed_Summary:
    #     Preprocessor.Save_Passed(word, pass_sum_path)

    # for word in removed_Summary:
    #     Preprocessor.Save_Removed(word, removed_sum_path)
        
    final_records_ct+=1

processed_df = pd.DataFrame(processed_data)
processed_df.to_csv(processed_records_path, index=False)
print("Processed Dataset Records: ", final_records_ct)

# processed_df = pd.DataFrame(preprocessed_qalsadi)
# processed_df.to_csv(processed_records_Qalpath, index=False)
print(processed_df)

KeyboardInterrupt: 

In [ ]:
processed_df

,paragraph,summary
0,تحت عنوان كارثة تحدى أبدأ كاتب عرض كتاب رابع و...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,اعترف دبلوماسي هاتين دولة عريض وقع أعيان اعتبر...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,قام ولاية حلب اعلان جنرال فرنسة هنري غور ١٩٢٠ ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,دول مصر العربيه ليست دول ليست شعب لابل مصرالار...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,سوري أصر استقلال بلاد : مثلما رفض سوري سيطرة ع...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...
...,...,...
149,حزب وفد احتفل مئوي ثورة ١٩١٩ ٩ قادم ملك شعب مص...,احتفال مئوية ثورة 1919 كان من منطلق وطني ليس ح...
150,علن مجلس قياد ثورة ١٨ يونيه ١٩٥٣ قيام جمهور مص...,مجلس قيادة الثورة أعلن عن قيام الجمهورية المصر...
151,رغم عبد رحمن فهم ضام جهاز سر عدد كبير شباب حلق...,ضم عبد الرحمن فهمي في الجهاز السري عدد كبير من...
152,اقتصر مقوم بورسعيد مدين عالم نشأة تمتع سائل مد...,امتدت بورسعيد لكي تشمل الطابع الثقافي للمدينة،...


In [ ]:
print(type(processed_df))

<class 'pandas.core.frame.DataFrame'>


## Model Building

In [211]:
# Assuming your DataFrame has columns 'text' and 'summary'
data = pd.read_csv("D:/NLP competition/Arabic-Text-Summarizer-v2.0/Debug/Preprocessed Dataset/Processed_Dataset.csv") 
data

,paragraph,summary
0,تحت عنوان كارثة تحدى أبدأ كاتب عرض كتاب رابع و...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,اعترف دبلوماسي هاتين دولة عريض وقع أعيان اعتبر...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,قام ولاية حلب اعلان جنرال فرنسة هنري غور ١٩٢٠ ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,دول مصر العربيه ليست دول ليست شعب لابل مصرالار...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,سوري أصر استقلال بلاد : مثلما رفض سوري سيطرة ع...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...
...,...,...
149,حزب وفد احتفل مئوي ثورة ١٩١٩ ٩ قادم ملك شعب مص...,احتفال مئوية ثورة 1919 كان من منطلق وطني ليس ح...
150,علن مجلس قياد ثورة ١٨ يونيه ١٩٥٣ قيام جمهور مص...,مجلس قيادة الثورة أعلن عن قيام الجمهورية المصر...
151,رغم عبد رحمن فهم ضام جهاز سر عدد كبير شباب حلق...,ضم عبد الرحمن فهمي في الجهاز السري عدد كبير من...
152,اقتصر مقوم بورسعيد مدين عالم نشأة تمتع سائل مد...,امتدت بورسعيد لكي تشمل الطابع الثقافي للمدينة،...


In [228]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

In [229]:
class MyDataset(Dataset):
    def __init__(self, paragraphs, summaries):
        self.paragraphs = paragraphs
        self.summaries = summaries

    def __len__(self):
        return len(self.paragraphs)

    def __getitem__(self, idx):
        source_text = self.paragraphs[idx]
        target_text = self.summaries[idx]
        return source_text, target_text

In [230]:
# Step 2: Tokenize and Preprocess the Dataset
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

In [231]:
def preprocess_data(text, summary):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    labels = tokenizer(summary, truncation=True, padding=True, max_length=150, return_tensors="pt")
    input_ids = inputs["input_ids"].squeeze()
    attention_mask = inputs["attention_mask"].squeeze()
    labels = labels["input_ids"].squeeze()
    return input_ids, attention_mask, labels

In [232]:
# Step 3: Split the data into Train and Test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [233]:
# Step 4: Define the Fine-Tuning Model
model = AutoModelForCausalLM.from_pretrained("UBC-NLP/MARBERT")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [266]:
def fine_tune(model, dataloader, num_epochs=1, learning_rate=1e-5):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(dataloader, desc="Epoch {}/{}".format(epoch + 1, num_epochs), leave=False)
        
        for batch in progress_bar:
            input_ids = []
            attention_mask = []

            for item in batch:
                paragraph = item[0]
                summary = item[1]
                
                # Tokenize the paragraph and summary
                inputs = tokenizer.encode_plus(paragraph, summary, add_special_tokens=True, padding="max_length", max_length=512, truncation=True)
                input_ids.append(inputs["input_ids"])
                attention_mask.append(inputs["attention_mask"])
            
            input_ids = torch.tensor(input_ids)
            attention_mask = torch.tensor(attention_mask)
            labels = None

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device) if labels is not None else None

            model.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            if loss is not None:
                total_loss += loss.item()

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

            progress_bar.set_postfix({"loss": total_loss / len(dataloader)})


In [267]:
def evaluate(model, val_dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            num_correct += (predicted_labels == labels).sum().item()
            num_samples += labels.size(0)

    accuracy = num_correct / num_samples
    print("Validation Accuracy: {:.4f}".format(accuracy))


In [268]:
# Step 6: Save and Use the Fine-Tuned Model
def save_model(model, output_dir):
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer


In [269]:
print(train_data)

                                             paragraph  \
118  نتائج تاريخ هزيم صليبي ١٠٠ رسالة جوي ( كوى ) ب...   
69   خلاف اعتقاد سائد تاريخ أرستقراطي ألقاب أمر ناد...   
26   حمد منصور : سلام على رحمة الله بركة حي هواء مب...   
132  مراد سئم قتال فرنسي صعيد معتاد نوع معيشة خشن ب...   
146  عند قيام حرب فلسطين ١٩٤٨ ساهم عزيز المصرى باش ...   
..                                                 ...   
71   ساع بريطاني أسباب متعلق أزمة أثناء بعد حرب عال...   
106  بلغة رسالة دخول مصر رجع إلا سر بركة الله بعث إ...   
14   صيف ١٩٦٨ شكل قياد دفاع جوي مصر تحمل المسئولية ...   
92   وفى صلاح دين دمشق ٥٨٩ ١١٩٣ خلف أخو عادل سقط دل...   
102  نتيجة خطر مغول فكر خليف مستنصر الله استعانة با...   

                                               summary  
118  واجهت الحملة الصليبية على مصر نهايتها المأساوي...  
69    في أوائل العصور الوسطى في الغرب أصبحت  الألقا...  
26   عدد الجوعى في العالم يزداد سنويا بمعدل خمسة مل...  
132  مراد سئم القتال ضد الفرنسيين لأنه لم يكن معتاد...  
146  لما قامت حرب 

In [270]:
train_paragraphs = [example[0] for example in train_data]
train_summaries = [example[1] for example in train_data]
test_paragraphs = [example[0] for example in test_data]
test_summaries = [example[1] for example in test_data]


In [271]:
# Step 9: Create Train and Test Datasets
train_dataset = MyDataset(train_paragraphs, train_summaries)
test_dataset = MyDataset(test_paragraphs, test_summaries)


In [272]:
batch_size = 32

In [273]:
print(type(train_dataset))
type(test_dataset)

<class '__main__.MyDataset'>


__main__.MyDataset

In [274]:
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [275]:
# Step 11: Fine-Tune the Model on Train data
fine_tune(model, train_dataloader)

c:\Users\steve\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/1:   0%|          | 0/1 [00:00<?, ?it/s]

In [276]:
# Step 12: Evaluate the Model on Test data
evaluate(model, test_dataloader)

# Save the fine-tuned model
output_dir = "./fine_tuned_model"
save_model(model, output_dir)

# Load the fine-tuned model
model, tokenizer = load_model(output_dir)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Generate summaries using the fine-tuned model
input_text = data["paragraph"]
inputs = tokenizer(input_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
generated_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_summaries)